In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, FloatType, StructField, StructType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import *

spark = SparkSession.builder.appName('MLflow Model inference example ').getOrCreate()

cols=("type","quality")
df=spark.read.option("header",True).option("inferSchema",True).csv('/home/cdsw/winequalityN.csv').drop(*cols)
df.show()
df.count()
#rdd.toDF().show()
df.printSchema()

Setting spark.hadoop.yarn.resourcemanager.principal to hrongali


+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+
|          7.0|            0.27|       0.36|          20.7|    0.045|               45.0|               170.0|  1.001| 3.0|     0.45|    8.8|
|          6.3|             0.3|       0.34|           1.6|    0.049|               14.0|               132.0|  0.994| 3.3|     0.49|    9.5|
|          8.1|            0.28|        0.4|           6.9|     0.05|               30.0|                97.0| 0.9951|3.26|     0.44|   10.1|
|          7.2|            0.23|       0.32|           8.5|    0.058|               47.0|               186.0| 0.9956|3.19|      0.4|    9.9|
|     

In [13]:
import mlflow
import mlflow.pyfunc
import pandas as pd
from pyspark.sql.functions import struct
print(mlflow.__version__)
import sklearn

1.19.0


In [14]:
predict = mlflow.pyfunc.spark_udf(spark, "/home/cdsw/mlmodel/model")

In [15]:
samp_df=df.sample(withReplacement=False, fraction=0.005,seed=42)

In [16]:
samp_df.withColumn("prediction", predict(struct("fixed acidity", "volatile acidity","citric acid","residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide","density","pH","sulphates","alcohol"))).show()

#samp_df.withColumn("prediction", predict(struct("fixed acidity", "volatile acidity"))).show()

+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+------------------+
|fixed acidity|volatile acidity|citric acid|residual sugar|chlorides|free sulfur dioxide|total sulfur dioxide|density|  pH|sulphates|alcohol|        prediction|
+-------------+----------------+-----------+--------------+---------+-------------------+--------------------+-------+----+---------+-------+------------------+
|          6.7|            0.26|       0.39|           1.1|     0.04|               45.0|               147.0| 0.9935|3.32|     0.58|    9.6|5.1762280178914395|
|          5.7|            0.36|       0.21|           6.7|    0.038|               51.0|               166.0| 0.9941|3.29|     0.63|   10.0| 5.139524461297263|
|          7.4|            0.25|       0.37|           6.9|     0.02|               14.0|                93.0| 0.9939| 3.0|     0.48|   10.7|5.3928896389154515|
|          6.3|            0.21|  